In [2]:
# import libraries
import cv2
import numpy as np
import mediapipe as mp
import os
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.callbacks import TensorBoard

In [3]:
mp_holistic = mp.solutions.holistic # holistic model
mp_drawing = mp.solutions.drawing_utils # Initialize mediapipe drawing utilities

In [4]:
def detection (img, model):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # convert the input into RGB format to pass to mediapipe
    img.flags.writeable = False # To improve performance, mark the image as not writeable
    results = model.process(img) # detection model
    img.flags.writeable = True
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR) # convert the input into BGR format to pass to opencv imshow
    return img, results

In [5]:
def draw_landmarks (img, results):
    mp_drawing.draw_landmarks(img, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, mp_drawing.DrawingSpec(color=(255,128,0), thickness=2, circle_radius=2),mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=2))
    mp_drawing.draw_landmarks(img, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, mp_drawing.DrawingSpec(color=(255,51,153), thickness=2, circle_radius=2),mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=2))
    mp_drawing.draw_landmarks(img, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, mp_drawing.DrawingSpec(color=(255,51,153), thickness=2, circle_radius=2),mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=2))

In [6]:
# get webcam
cap = cv2.VideoCapture('./videoplayback.mp4')
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while True:
        ret, frame = cap.read()

        img, results = detection(frame, holistic) # make detectionq
        draw_landmarks(img,results) # draw landmarks

        cv2.imshow('Human Pose Estimation', img)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

In [7]:
# extracting body keypoints
# flatten the array so that it can be passed to the LSTM model
def keypoint_extraction(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    left_hand = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    right_hand = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, left_hand, right_hand])

In [8]:
DATA_PATH = os.path.join('MP_Data') 
actions = ['punch', 'guard', 'kick', 'normal'] # declare actions
vids_num = 30
vid_length = 30 #frames

# data directory
for action in actions:
    for vid in range(vids_num):
        os.makedirs(os.path.join('data', action, str(vid)))

In [ ]:
# collecting data
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    for action in actions:
        for vid in range(vids_num):
            for frame_num in range(vid_length):
                ret, frame = cap.read()

                img, results = detection(frame, holistic)
                draw_landmarks(img,results) 

                if frame_num == 0: 
                    cv2.putText(img, 'Start collecting', (120,200), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(img, 'Collecting frames for {} Video Number {}'.format(action, vid), (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    cv2.imshow('OpenCV Feed', img)
                    cv2.waitKey(2000)
                else: 
                    cv2.putText(img, 'Collecting frames for {} Video Number {}'.format(action, vid), (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    cv2.imshow('OpenCV Feed', img)
                
                keypoints = keypoint_extraction(results)
                npy_path = os.path.join(DATA_PATH, action, str(vid), str(frame_num))
                np.save(npy_path, keypoints)

                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
    cap.release()
    cv2.destroyAllWindows()

In [ ]:
labels = {label:num for num, label in enumerate(actions)}

In [ ]:
sequences, labels = [], []
for action in actions:
    for vid in range(vids_num):
        window = []
        for frame_num in range(vid_length):
            res = np.load(os.path.join(DATA_PATH, action, str(vid), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(labels[action])

print(np.array(sequences).shape)
print(np.array(labels).shape)

In [ ]:
# dataset
X = np.array(sequences)
y = to_categorical(labels).astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [ ]:
# LSTM model
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [ ]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)
model.summary()

In [ ]:
model.fit(X_train, y_train, epochs=500, callbacks=[tb_callback])

In [ ]:
# predictions
res = model.predict(X_test)
print(actions[np.argmax(res[4])])
print(actions[np.argmax(y_test[4])])

In [ ]:
model.save('action.h5')